In [ ]:
! pip install osmnx jupyter sumo-user-libs selenium webdriver-manager

In [ ]:
! sudo apt install netconvert

In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt

ox.settings.all_oneway = True

# Let's try a different, well-known area.
place_name = "Berkeley, California, USA"

graph_unsimplified = ox.graph_from_place(place_name, network_type='drive', simplify=False)

ox.save_graph_xml(graph_unsimplified, filepath="./map.osm.xml")
print("Unsimplified graph successfully saved as map.osm.xml")


fig, ax = ox.plot_graph(graph_unsimplified)
plt.show()

KeyboardInterrupt: 

In [10]:
! netconvert --osm-files map.osm.xml -o map.net.xml

netconvert: ./src/netbuild/NBAlgorithms.h:193: double NBNodesEdgesSorter::edge_by_junction_angle_sorter::getConvAngle(NBEdge*) const: Assertion `angle >= 0 && angle < (double)360' failed.
